#Downloading the labeled and unlabeled data

In [1]:
!gdown https://drive.google.com/uc?id=18MA0qKg1rqP92HApr_Fjck7Zo4Bwdqdu
!unzip HV-AI-2025.zip
!rm -rf /content/__MACOSX
!mv /content/HV-AI-2025/* /content/
!rm -rf /content/HV-AI-2025
!rm /content/HV-AI-2025.zip
!rm -rf /content/sample_data
from google.colab import output
output.clear()

#Downloading the test data

In [3]:
!gdown https://drive.google.com/uc?id=1aszVlQFQOwJTy9tt79s7x87VJyYw-Sxy
!unzip HV-AI-2025-Test.zip
!rm -rf /content/__MACOSX
!mv /content/HV-AI-2025-Test/* /content/
!rm -rf /content/HV-AI-2025-Test
!rm /content/HV-AI-2025-Test.zip
!rm -rf /content/sample_data
from google.colab import output
output.clear()

# **Load/Preprocess **data****

In [1]:
# 📦 Imports & Config
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

# Set Device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")


Using device: cpu


In [5]:
# 📊 Load labels CSV and create label <-> index mapping
labels_df = pd.read_csv(r'C:\Users\cmane\Downloads\labeled_data.csv')
unique_labels = sorted(labels_df['label'].unique())
label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

print("Label to Index mapping:", label_to_idx)


Label to Index mapping: {'cane': 0, 'cavallo': 1, 'elefante': 2, 'farfalla': 3, 'gallina': 4, 'gatto': 5, 'mucca': 6, 'pecora': 7, 'ragno': 8, 'scoiattolo': 9}


In [6]:
# 🗂️ Custom Dataset for labeled data
class ImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, label_to_idx=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.label_to_idx = label_to_idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.data.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")

        label_str = self.data.iloc[idx, 1]
        label = self.label_to_idx[label_str]

        if self.transform:
            image = self.transform(image)

        return image, label


In [7]:
# 📦 Data augmentation and normalization
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [9]:
# 📦 DataLoader setup
train_dataset = ImageDataset(
    r"C:\Users\cmane\Downloads\labeled_data.csv",
    r"C:\Users\cmane\Downloads\images",
    transform=train_transform,
    label_to_idx=label_to_idx
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


# **Model initialization/Training**

In [10]:
# 🧠 Load pretrained ResNet-18 and modify for 10 classes
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(DEVICE)


c:\Users\cmane\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\cmane\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\cmane/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:17<00:00, 2.68MB/s]


In [11]:
# 🎯 Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)


In [12]:
# 🚀 Training loop
EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    print(f"Epoch [{epoch+1}/{EPOCHS}] - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")


100%|██████████| 25/25 [00:40<00:00,  1.62s/it]


Epoch [1/5] - Loss: 1.2874, Accuracy: 57.25%


100%|██████████| 25/25 [00:31<00:00,  1.25s/it]


Epoch [2/5] - Loss: 0.9346, Accuracy: 70.22%


100%|██████████| 25/25 [00:27<00:00,  1.08s/it]


Epoch [3/5] - Loss: 0.7536, Accuracy: 73.81%


100%|██████████| 25/25 [00:27<00:00,  1.11s/it]


Epoch [4/5] - Loss: 0.7355, Accuracy: 74.58%


100%|██████████| 25/25 [00:27<00:00,  1.10s/it]

Epoch [5/5] - Loss: 0.6082, Accuracy: 79.85%


In [13]:
import time

EPOCHS = 10  # Increased to 10
best_acc = 0.0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    start_time = time.time()

    for images, labels in tqdm(train_loader):
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    elapsed_time = time.time() - start_time

    print(f"Epoch [{epoch+1}/{EPOCHS}] - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%, Time: {elapsed_time:.2f}s")

    # Optional: Track best accuracy so far
    if epoch_acc > best_acc:
        best_acc = epoch_acc

print(f"\n✅ Training complete. Best accuracy achieved: {best_acc:.2f}%")


100%|██████████| 25/25 [00:25<00:00,  1.01s/it]


Epoch [1/10] - Loss: 0.5456, Accuracy: 81.64%, Time: 25.35s


100%|██████████| 25/25 [00:27<00:00,  1.08s/it]


Epoch [2/10] - Loss: 0.5329, Accuracy: 82.28%, Time: 27.06s


100%|██████████| 25/25 [00:27<00:00,  1.11s/it]


...

Epoch [8/10] - Loss: 0.4950, Accuracy: 82.93%, Time: 28.09s


100%|██████████| 25/25 [00:28<00:00,  1.13s/it]


Epoch [9/10] - Loss: 0.3945, Accuracy: 87.03%, Time: 28.37s


100%|██████████| 25/25 [00:28<00:00,  1.14s/it]

Epoch [10/10] - Loss: 0.4135, Accuracy: 87.29%, Time: 28.59s

✅ Training complete. Best accuracy achieved: 87.29%


# **Model Inference**

In [14]:
# 📸 Dataset class for test images (no labels)
class TestDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.img_names = sorted(os.listdir(img_dir))
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.img_names[idx]


In [15]:
# 📦 DataLoader for test images
test_dataset = TestDataset(r"C:\Users\cmane\Downloads\test_images", transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [16]:
# 📊 Run inference and save predictions
model.eval()
predictions = []

with torch.no_grad():
    for images, img_names in tqdm(test_loader):
        images = images.to(DEVICE)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        preds_str = [idx_to_label[p.item()] for p in preds]

        for img_name, label in zip(img_names, preds_str):
            predictions.append({'path': img_name, 'predicted_label': label})

# 📄 Save predictions to CSV
submission_df = pd.DataFrame(predictions)
submission_df.to_csv('phase1_predictions.csv', index=False)

print("✅ Predictions saved to 'phase1_predictions.csv'")


100%|██████████| 328/328 [04:47<00:00,  1.14it/s]

✅ Predictions saved to 'phase1_predictions.csv'


In [17]:
# Save model state dict after training
torch.save(model.state_dict(), 'resnet18_phase1.pth')


PHASE 2

In [18]:
import os
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import pandas as pd
from tqdm import tqdm
import numpy as np


In [19]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [20]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])


In [21]:
class ImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, label_to_idx=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.label_to_idx = label_to_idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.data.iloc[idx, 0])
        image = Image.open(img_path).convert('RGB')
        label = self.label_to_idx[self.data.iloc[idx, 1]]

        if self.transform:
            image = self.transform(image)

        return image, label

class UnlabeledDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.image_names = os.listdir(img_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_names[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.image_names[idx]


In [22]:
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
label_to_idx = {'class_0':0, 'class_1':1, 'class_2':2, 'class_3':3, 'class_4':4,
                'class_5':5, 'class_6':6, 'class_7':7, 'class_8':8, 'class_9':9}
inv_label_map = {v: k for k, v in label_to_idx.items()}

model.fc = nn.Linear(num_features, len(label_to_idx))
model.load_state_dict(torch.load('resnet18_phase1.pth'))
model = model.to(DEVICE)
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [23]:
unlabeled_dataset = UnlabeledDataset(r"C:\Users\cmane\Downloads\images2", transform=test_transform)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=32, shuffle=False)

pseudo_labels = []
confidence_threshold = 0.8

with torch.no_grad():
    for images, image_names in tqdm(unlabeled_loader):
        images = images.to(DEVICE)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        max_probs, preds = torch.max(probs, dim=1)

        for i in range(len(image_names)):
            if max_probs[i] >= confidence_threshold:
                pseudo_labels.append((image_names[i], preds[i].item()))

print(f"Pseudo-labeled {len(pseudo_labels)} images with confidence >= {confidence_threshold}")


100%|██████████| 463/463 [05:59<00:00,  1.29it/s]

Pseudo-labeled 8330 images with confidence >= 0.8


In [31]:
pseudo_dataset_images = []
pseudo_dataset_labels = []

with torch.no_grad():
    for images, image_names in tqdm(unlabeled_loader):
        images = images.to(DEVICE)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        max_probs, preds = torch.max(probs, dim=1)

        for i in range(len(image_names)):
            if max_probs[i] >= confidence_threshold:
                pseudo_dataset_images.append(os.path.join('./dataset/unlabeled_data', image_names[i]))
                pseudo_dataset_labels.append(preds[i].item())

print(f"Pseudo-labeled {len(pseudo_dataset_images)} images")


100%|██████████| 463/463 [04:25<00:00,  1.75it/s]

Pseudo-labeled 7758 images


In [32]:
class PseudoLabeledDataset(Dataset):
    def __init__(self, img_paths, labels, transform=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


In [33]:
# Labeled dataset
labeled_dataset = ImageDataset(
    csv_file=r"C:\Users\cmane\Downloads\labeled_data.csv",
    img_dir=r"C:\Users\cmane\Downloads\images",
    transform=train_transform,
    label_to_idx=label_to_idx
)

# Pseudo-labeled dataset
pseudo_dataset = PseudoLabeledDataset(
    img_paths=pseudo_dataset_images,
    labels=pseudo_dataset_labels,
    transform=train_transform
)

# Combine them
combined_dataset = torch.utils.data.ConcatDataset([labeled_dataset, pseudo_dataset])

# Final DataLoader
combined_loader = DataLoader(combined_dataset, batch_size=32, shuffle=True)


In [35]:
model.eval()
predictions = []

with torch.no_grad():
    for images, image_names in tqdm(test_loader):
        images = images.to(DEVICE)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        for img_name, pred in zip(image_names, preds):
            predictions.append([img_name, inv_label_map[pred.item()]])

submission_df = pd.DataFrame(predictions, columns=['path', 'predicted_label'])
submission_df.to_csv('phase2_predictions.csv', index=False)



100%|██████████| 328/328 [05:27<00:00,  1.00it/s]


# **Helper Functions**

In [ ]:
import requests

def send_results_for_evaluation(name, csv_file, email):
    url = "http://43.205.49.236:5050/inference"
    files = {'file': open(csv_file, 'rb')}
    data = {'email': email, 'name':name}
    response = requests.post(url, files=files, data=data)
    return response.json()



# ***Test Inference***


This function is used to save the csv file and send it to the evaluation server.

Format of CSV file (Follow the header names strictly):

          path (str)               predicted_label(str)
    test_data/images/xx.jpg               class_1         
    test_data/images/yy.jpg               class_2         
             :                               :                          
             :                               :                          

Once the prediction file is saved as shown in the above format, you can send it to the evaluation server along with your email.

Caution: check your **email** before executing the cell.


In [38]:

print('Accuracy: ')
print(send_results_for_evaluation('C.Maneesh Kumar Reddy', r"C:\Users\cmane\Downloads\phase1_predictions.csv", 'goldenprince1436@gmail.com'))




Accuracy: 


NameError: name 'send_results_for_evaluation' is not defined